In [1]:
# demo_full_flow.py
# Demo completa: RESHARE -> INPUT -> MULTIPLY -> OUTPUT
# Requisitos: ejecutar en entorno Sage (sageenv). Usa AuthShare y SimNetwork.
#
# Comentarios y explicaciones en español dentro del código.

from sage.all import GF
from typing import List, Tuple, Dict, Any

# ---------- Copia (o importa) la clase AuthShare, make_rand_auth y SimNetwork ----------
class AuthShare:
    """
    Representa una compartición autenticada <x>:
    - F: campo (GF(p))
    - delta: delta asociado (puede ser 0 si no se usa)
    - shares: lista de shares locales
    - macs: lista de macs locales
    """
    def __init__(self, F, delta, shares: List[Any], macs: List[Any]):
        self.F = F
        self.delta = F(delta)
        self.shares = [F(s) for s in shares]
        self.macs = [F(m) for m in macs]

    def value(self):
        # reconstrucción del secreto (suma de shares)
        return sum(self.shares)

    def mac_sum(self):
        return sum(self.macs)

    def n_players(self):
        return len(self.shares)


def make_rand_auth(F, v, alpha, n):
    """
    Construye un AuthShare 'consistente' para pruebas:
      - v: elemento del campo (valor secreto)
      - alpha: clave global
      - n: número de jugadores
    Devuelve AuthShare con sum(shares)=v y sum(macs)=alpha * v.
    """
    shares = [F.random_element() for _ in range(n-1)]
    shares.append(F(v - sum(shares)))
    delta = F(0)
    mac_target = alpha * v
    macs = [F.random_element() for _ in range(n-1)]
    macs.append(F(mac_target - sum(macs)))
    return AuthShare(F, delta, shares, macs)


class SimNetwork:
    """
    Red simulada muy simple para testing local.
    - start_round() limpia el buffer
    - broadcast(sender_id, value) añade (sender_id, value)
    - collect() devuelve la lista de mensajes
    - end_round() no hace nada aquí
    """
    def __init__(self, n):
        self.n = n
        self._buffer = []

    def start_round(self):
        self._buffer = []

    def broadcast(self, sender_id: int, value):
        self._buffer.append((sender_id, value))

    def collect(self):
        # devuelve lo enviado en la ronda actual
        return list(self._buffer)

    def end_round(self):
        pass

# ----------------- Funciones online corregidas -----------------

def verify_authshare_consistency(auth: AuthShare, alpha) -> bool:
    """
    Comprueba que sum(macs) == alpha * sum(shares).
    """
    return int(auth.mac_sum()) == int(alpha * auth.value())


def reshare_practical(F, X_auth: AuthShare, R_auth: AuthShare, alpha, network: SimNetwork,
                      preserve_secret: bool = True) -> Tuple[AuthShare, Dict]:
    """
    Reshare práctico (opción A). Usa R (del preprocesamiento) para 'reenvolver' X.
    - Si preserve_secret=True: new_auth.value() == original x (no se introduce delta).
    - Si preserve_secret=False: new_auth.value() == x + delta_new (delta aleatorio).
    Simula la comunicación con 'network' (broadcast de d_i = x_i - r_i).
    Retorna (new_auth, transcript).
    """
    n = X_auth.n_players()
    assert n == R_auth.n_players()

    network.start_round()
    # 1) cada jugador publica d_i = x_i - r_i
    d_i = [X_auth.shares[i] - R_auth.shares[i] for i in range(n)]
    for i in range(n):
        network.broadcast(i, d_i[i])

    # 2) reconstruir eps = suma d_i
    msgs = network.collect()
    eps = sum(v for (_, v) in msgs)
    eps = F(eps)

    # 3) reconstruimos x = r + eps (r = R_auth.value())
    r = R_auth.value()
    x = r + eps

    # 4) generamos nuevo AuthShare consistente
    if preserve_secret:
        delta_new = F(0)
        target = x
    else:
        delta_new = F.random_element()
        target = x + delta_new

    # construir n-1 shares aleatorios y ajustar el último para sumar 'target'
    new_shares = [F.random_element() for _ in range(n-1)]
    new_shares.append(F(target - sum(new_shares)))

    # construir macs consistentes: sum(macs) == alpha * target
    mac_target = alpha * target
    new_macs = [F.random_element() for _ in range(n-1)]
    new_macs.append(F(mac_target - sum(new_macs)))

    network.end_round()

    new_auth = AuthShare(F, delta_new, new_shares, new_macs)
    transcript = {
        "round_msgs": [(int(sid), int(v)) for (sid, v) in msgs],
        "eps": int(eps),
        "target_reconstructed": int(target)
    }
    return new_auth, transcript


def input_phase(F, players, x_input, R_auth: AuthShare, alpha, network: SimNetwork) -> Tuple[AuthShare, Dict]:
    """
    Protocol de entrada (input):
    - R_auth: AuthShare de un 'mask' r procedente del preprocesamiento para esta entrada.
    - x_input: valor del cliente (elemento en F)
    Procedimiento:
      1. Se abre eps = x - r (el cliente publica eps).
      2. El cliente reparte EPS en shares (epsilon_shares) y cada jugador añade su epsilon_share
         a su share local de R (de esta manera se obtiene una AuthShare de x).
      3. Se genera la nueva AuthShare X con macs consistentes.
    Retorna (X_auth, transcript).
    """
    n = R_auth.n_players()
    network.start_round()

    # 1) reconstruir r (lo hace cliente / servidor del preproc; en demo lo computamos)
    r = R_auth.value()
    eps = F(x_input - r)   # valor público que el cliente publica

    # 2) el cliente crea una partición aleatoria de eps en n shares:
    eps_shares = [F.random_element() for _ in range(n - 1)]
    eps_shares.append(F(eps - sum(eps_shares)))   # último share para cerrar la suma

    # cliente "envía" eps_shares[i] a jugador i (simulado con broadcast)
    for i in range(n):
        network.broadcast(i, eps_shares[i])

    msgs = network.collect()
    # comprobación (opcional): suma de eps_shares == eps
    eps_recon = sum(v for (_, v) in msgs)
    assert int(eps_recon) == int(eps), "Error: eps_shares no suman eps"

    # 3) cada jugador construye su share de X: x_i = r_i + eps_share_i
    #    y corrige su mac local: mac_i' = r_mac_i + eps_share_i * alpha
    new_shares = []
    new_macs = []
    for i in range(n):
        e_i = msgs[i][1]
        new_shares.append(F(R_auth.shares[i] + e_i))
        new_macs.append(F(R_auth.macs[i] + (e_i * alpha)))

    # En teoría la sum(new_macs) = alpha * (r + sum(eps_shares)) = alpha * x_input
    network.end_round()

    X_auth = AuthShare(F, R_auth.delta, new_shares, new_macs)
    transcript = {
        "eps_opened": int(eps),
        "eps_shares": [int(v) for (_, v) in msgs]
    }
    return X_auth, transcript


def multiply_phase(F, X_auth: AuthShare, Y_auth: AuthShare, triple: Tuple[AuthShare, AuthShare, AuthShare],
                   alpha, network: SimNetwork) -> Tuple[AuthShare, Dict]:
    """
    Multiplicación usando triple de Beaver (A,B,C).
    Input: X_auth, Y_auth compartidos, triple = (A,B,C) (AuthShare cada uno).
    Output: Z_auth (AuthShare de Z = X*Y), transcript con d,e abiertos.
    Algoritmo (versión simplificada y consistente con MACs):
      1. Cada jugador localmente computa d_i = X_i - A_i, e_i = Y_i - B_i; broadcast.
      2. Reconstruir d = sum d_i, e = sum e_i (valores públicos).
      3. Construir Z_share_i = C_i + d * B_i + e * A_i
         Además, añadir d*e al último share para ajustar la suma (d*e es público).
      4. Construir macs correspondientemente y ajustar el último mac para que
         sum(macs) == alpha * Z_value.
    """
    A, B, C = triple
    n = X_auth.n_players()
    assert n == Y_auth.n_players() == A.n_players() == B.n_players() == C.n_players()

    network.start_round()

    # (1) cada jugador publica d_i, e_i
    d_i = [X_auth.shares[i] - A.shares[i] for i in range(n)]
    e_i = [Y_auth.shares[i] - B.shares[i] for i in range(n)]
    for i in range(n):
        # empaquetamos como (d_i, e_i) por simplicidad en el buffer
        network.broadcast(i, (d_i[i], e_i[i]))

    msgs = network.collect()
    # msgs: lista de (sender_id, (d_i, e_i))
    # reconstruir d,y e
    d = sum(pair[1][0] for pair in msgs)
    e = sum(pair[1][1] for pair in msgs)
    d = F(d)
    e = F(e)

    # (2) construir Z shares
    new_shares = []
    for i in range(n):
        term1 = C.shares[i]
        term2 = d * B.shares[i]    # d * (B_i)
        term3 = e * A.shares[i]    # e * (A_i)
        new_shares.append(F(term1 + term2 + term3))

    # añadir d*e (público) al último share para que sum(new_shares) == Z_value
    new_shares[-1] = F(new_shares[-1] + (d * e))

    # (3) construir macs: start from C.macs + d*B.macs + e*A.macs
    new_macs = []
    for i in range(n):
        mterm = C.macs[i] + d * B.macs[i] + e * A.macs[i]
        new_macs.append(F(mterm))

    # ahora corregimos la suma de macs para que sea = alpha * Z_value
    # calcular Z_value de forma explícita (reconstrucción "real")
    Z_value = sum(new_shares)
    mac_target = alpha * Z_value
    # ajustar último mac
    new_macs[-1] = F(mac_target - sum(new_macs[:-1]))

    network.end_round()
    Z_auth = AuthShare(F, C.delta, new_shares, new_macs)

    transcript = {
        "opened_d": int(d),
        "opened_e": int(e),
        "Z_value": int(Z_value)
    }
    return Z_auth, transcript


def output_phase(F, opened_list: List[AuthShare], Z_auth: AuthShare, alpha) -> Tuple[bool, Any]:
    """
    Output / verificación:
    - opened_list: lista de AuthShare que se abrieron durante la ejecución (por ejemplo inputs)
                   (en esta demo asumimos que ya están "abiertos", es decir, se puede llamar .value()).
    - Z_auth: AuthShare del resultado a verificar/abrir
    - alpha: clave global
    Retorna: (ok, z_value) donde ok indica si macs coinciden y z_value es Z mod p.
    """
    # reconstruimos Z
    z_val = Z_auth.value()
    mac_sum = Z_auth.mac_sum()

    ok = (int(mac_sum) == int(alpha * z_val))
    return ok, int(z_val)


# ----------------- Demo que enlaza todo -----------------

def demo_full_flow(preproc):
    """
    preproc: diccionario con:
      - 'R_x': AuthShare (mask para input X)
      - 'R_y': AuthShare (mask para input Y)
      - 'triple': (A,B,C) AuthShare triple de Beaver
      - 'alpha': clave global (elemento de F)
      - 'F': campo GF(p)
    Ejecuta: reshare (opcional), input(X), input(Y), multiply, output.
    """
    F = preproc['F']
    alpha = preproc['alpha']
    R_x = preproc['R_x']
    R_y = preproc['R_y']
    triple = preproc['triple']
    n = R_x.n_players()

    net = SimNetwork(n)

    print("\n=== (0) verificación preprocesamiento ===")
    print("R_x OK:", verify_authshare_consistency(R_x, alpha))
    print("R_y OK:", verify_authshare_consistency(R_y, alpha))
    print("A,B,C OK:", all(verify_authshare_consistency(s, alpha) for s in triple))

    # (optional) ejemplo de Reshare para X (para mostrar enlace con preproc)
    print("\n=== (1) RESHARE práctico (re-envolver X si se necesitara) ===")
    # Simulamos que existe X_old y queremos reencapsularlo con otro R (a modo demo)
    # aquí creamos X_old a partir de un X_value (cliente): para demo, generamos un X_old compartido
    x_value = F(1234)
    X_old = make_rand_auth(F, x_value, alpha, n)
    newX, tr_reshare = reshare_practical(F, X_old, R_x, alpha, net, preserve_secret=True)
    print("Reshare transcript eps:", tr_reshare['eps'])
    assert verify_authshare_consistency(newX, alpha)

    # (2) INPUT phases: creamos X and Y a partir de los R_x y R_y (preproc)
    print("\n=== (2) INPUT phase para X y Y ===")
    # Supongamos que cliente desea inyectar x_input e y_input
    x_input = F(1234)
    y_input = F(55)

    X, trX = input_phase(F, None, x_input, R_x, alpha, net)
    Y, trY = input_phase(F, None, y_input, R_y, alpha, net)
    print("eps X abierto:", trX['eps_opened'])
    print("eps Y abierto:", trY['eps_opened'])
    assert verify_authshare_consistency(X, alpha)
    assert verify_authshare_consistency(Y, alpha)

    # (3) MULTIPLY
    print("\n=== (3) MULTIPLY phase ===")
    Z, trZ = multiply_phase(F, X, Y, triple, alpha, net)
    print("Opened d,e:", trZ['opened_d'], trZ['opened_e'])
    assert verify_authshare_consistency(Z, alpha)

    # (4) OUTPUT
    print("\n=== (4) OUTPUT phase ===")
    # en salida verificamos MACs de Z
    ok, zval = output_phase(F, [X, Y], Z, alpha)
    print("MAC correctos?:", ok)
    print("Resultado computado:", zval)
    print("Resultado esperado (x*y mod p):", int(x_input * y_input))
    return ok, zval


# ----------------- Ejecución de la demo con preprocesamiento simulado -----------------
if __name__ == "__main__":
    # Parámetros de prueba
    p = 2**61 - 1
    F = GF(p)
    n = 3
    alpha = F.random_element()

    # Preprocesamiento simulado: generamos R_x, R_y y triple A,B,C (todos consistentes con alpha)
    # En tu pipeline real esto viene del módulo de preprocesamiento.
    R_x = make_rand_auth(F, F.random_element(), alpha, n)
    R_y = make_rand_auth(F, F.random_element(), alpha, n)
    A = make_rand_auth(F, F.random_element(), alpha, n)
    B = make_rand_auth(F, F.random_element(), alpha, n)
    C = make_rand_auth(F, A.value() * B.value(), alpha, n)  # C debe ser a*b
    triple = (A, B, C)

    preproc = {'F': F, 'alpha': alpha, 'R_x': R_x, 'R_y': R_y, 'triple': triple}
    ok, zval = demo_full_flow(preproc)



=== (0) verificación preprocesamiento ===
R_x OK: True
R_y OK: True
A,B,C OK: True

=== (1) RESHARE práctico (re-envolver X si se necesitara) ===
Reshare transcript eps: 405692416089374753

=== (2) INPUT phase para X y Y ===
eps X abierto: 405692416089374753
eps Y abierto: 43204969419839897

=== (3) MULTIPLY phase ===
Opened d,e: 1393668724236996257 1810988779404915193

=== (4) OUTPUT phase ===
MAC correctos?: True
Resultado computado: 67870
Resultado esperado (x*y mod p): 67870
